# 🥗 Nutrition Assistant - Complete MLOps Pipeline

**Built for:** Albert School LLM OPS Bootcamp MSC2  
**Status:** ✅ Production Ready  
**Date Completed:** October 21, 2025  


---

## 📋 Table of Contents

1. [Project Overview & Architecture](#1)
2. [Environment Setup & Prerequisites](#2)
3. [GCP Configuration & Authentication](#3)
4. [Data Processing Pipeline](#4)
5. [Model Training with Vertex AI](#5)
6. [LoRA Fine-Tuning Configuration](#6)
7. [Model Deployment to Vertex AI](#7)
8. [Custom Handler Implementation](#8)
9. [Chainlit Chatbot Interface](#9)
10. [Cost Management & Monitoring](#10)
11. [Pipeline Components Deep Dive](#11)
12. [Troubleshooting Guide](#12)

---

## What This Project Does

This is an **end-to-end MLOps system** that:
- ✅ **Transforms** 2,395 nutrition items into conversational format
- ✅ **Fine-tunes** Microsoft Phi-3-mini model using LoRA (Low-Rank Adaptation)
- ✅ **Deploys** to Vertex AI endpoint with GPU acceleration
- ✅ **Serves** via a beautiful Chainlit web interface
- ✅ **Manages costs** with deploy/undeploy automation

# 1. Project Overview & Architecture 🏗️

## Complete System Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         DATA PROCESSING                                  │
│  ┌────────────────────────────────────────────────────────────────────┐ │
│  │ COMBINED_FOOD_DATASET.csv (2,395 food items)                       │ │
│  │ → Transform to conversational Q&A format                           │ │
│  │ → Train/Test split                                                 │ │
│  │ → Upload to GCS: gs://llmops_101_europ/data/                      │ │
│  └────────────────────────────────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────┐
│                    VERTEX AI TRAINING PIPELINE                           │
│  ┌──────────────┐   ┌──────────────┐   ┌──────────────┐  ┌───────────┐│
│  │ Component 1  │ → │ Component 2  │ → │ Component 3  │→ │Component 4││
│  │ Transform    │   │ Fine-Tune    │   │ Inference    │  │ Evaluate  ││
│  │              │   │ LoRA + 4-bit │   │ Generate     │  │ BLEU/     ││
│  │ CSV → JSON   │   │ Phi-3 model  │   │ Predictions  │  │ Rouge     ││
│  │              │   │ Tesla T4 GPU │   │              │  │           ││
│  └──────────────┘   └──────────────┘   └──────────────┘  └───────────┘│
│                                                                          │
│  Output: gs://llmops_101_europ/pipeline_root/.../fine_tuned_model/     │
└─────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────┐
│                      MODEL DEPLOYMENT                                    │
│  ┌────────────────────────────────────────────────────────────────────┐ │
│  │ 1. Register Model → Vertex AI Model Registry                      │ │
│  │    Model ID: 3561348948692041728                                  │ │
│  │                                                                     │ │
│  │ 2. Create Endpoint → nutrition-assistant-endpoint                 │ │
│  │    Endpoint ID: 5724492940806455296                               │ │
│  │                                                                     │ │
│  │ 3. Deploy Model → n1-standard-8 + Tesla T4 GPU                   │ │
│  │    Custom Handler: src/handler.py                                 │ │
│  │    Container: HuggingFace TGI PyTorch                            │ │
│  └────────────────────────────────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────┐
│                      CHAINLIT WEB INTERFACE                              │
│  ┌────────────────────────────────────────────────────────────────────┐ │
│  │ User → Chat UI (localhost:8000)                                   │ │
│  │   ↓                                                                │ │
│  │ Chainlit App (src/app/main.py)                                    │ │
│  │   ↓                                                                │ │
│  │ Google Cloud ADC Authentication                                   │ │
│  │   ↓                                                                │ │
│  │ Vertex AI Endpoint Prediction Request                            │ │
│  │   ↓                                                                │ │
│  │ Model Response (nutrition advice)                                 │ │
│  └────────────────────────────────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────────────────────┘
```

## Key Technologies

| Component | Technology | Purpose |
|-----------|------------|---------|
| **Model** | Microsoft Phi-3-mini-4k-instruct (3.8B params) | Base LLM |
| **Fine-Tuning** | LoRA (Low-Rank Adaptation) + 4-bit quantization | Efficient training |
| **Pipeline** | Vertex AI + Kubeflow v2.14.6 | Orchestration |
| **Compute** | Tesla T4 GPU + n1-standard-8 | Training & serving |
| **Storage** | Google Cloud Storage | Data & models |
| **Interface** | Chainlit + Python | User chat UI |
| **Auth** | Google Cloud ADC | Security |

In [ ]:
# Load project configuration
import sys
sys.path.append('.')

from src.constants import (
    GCP_PROJECT_ID, 
    GCP_REGION, 
    GCP_BUCKET_NAME,
    BASE_MODEL,
    TRAINING_CONFIG,
    LORA_CONFIG
)

print("📊 PROJECT CONFIGURATION")
print("=" * 60)
print(f"GCP Project ID:    {GCP_PROJECT_ID}")
print(f"GCP Region:        {GCP_REGION}")
print(f"GCS Bucket:        {GCP_BUCKET_NAME}")
print(f"Base Model:        {BASE_MODEL}")
print(f"Training Epochs:   {TRAINING_CONFIG['num_train_epochs']}")
print(f"Batch Size:        {TRAINING_CONFIG['per_device_train_batch_size']}")
print(f"Learning Rate:     {TRAINING_CONFIG['learning_rate']}")
print(f"LoRA Rank (r):     {LORA_CONFIG['r']}")
print(f"LoRA Alpha:        {LORA_CONFIG['lora_alpha']}")
print("=" * 60)

# 2. Environment Setup & Prerequisites 🔧

## Required Software

- **Python:** 3.11.6+
- **Package Manager:** `uv` (ultra-fast Python package installer)
- **Google Cloud SDK:** `gcloud` CLI
- **GPU Quota:** NVIDIA Tesla T4 in europe-west2

## Environment Validation

Run the following checks to ensure your environment is ready:

In [ ]:
# Check Python version
import sys
print(f"✅ Python Version: {sys.version}")

# Check installed packages
import subprocess

packages_to_check = ['google-cloud-aiplatform', 'chainlit', 'transformers', 'peft', 'bitsandbytes']
print("\n📦 Installed Packages:")
for package in packages_to_check:
    try:
        __import__(package.replace('-', '_'))
        print(f"  ✅ {package}")
    except ImportError:
        print(f"  ❌ {package} - NOT INSTALLED")

# Check gcloud CLI
try:
    result = subprocess.run(['gcloud', 'version'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print(f"\n✅ gcloud CLI installed")
        print(result.stdout.split('\n')[0])
    else:
        print("\n❌ gcloud CLI not found")
except Exception as e:
    print(f"\n❌ gcloud CLI not found: {e}")

# 3. GCP Configuration & Authentication 🔐

## Authentication Process

To work with Vertex AI and Google Cloud Storage, you need proper authentication:

### Method 1: Application Default Credentials (Recommended)

In [ ]:
# Test Google Cloud authentication
import google.auth
from google.cloud import aiplatform, storage

try:
    # Get default credentials
    credentials, project = google.auth.default()
    print(f"✅ Authentication successful!")
    print(f"📋 Project ID: {project}")
    print(f"👤 Credentials type: {type(credentials).__name__}")
    
    # Initialize Vertex AI
    aiplatform.init(
        project='aerobic-polygon-460910-v9',
        location='europe-west2'
    )
    print(f"✅ Vertex AI initialized")
    
    # Test GCS access
    storage_client = storage.Client()
    bucket = storage_client.bucket('llmops_101_europ')
    if bucket.exists():
        print(f"✅ GCS Bucket accessible: gs://llmops_101_europ")
    else:
        print(f"❌ GCS Bucket not found")
        
except Exception as e:
    print(f"❌ Authentication failed: {e}")
    print("\n💡 Run this in terminal:")
    print("   gcloud auth application-default login")

# 4. Data Processing Pipeline 📊

## Dataset Overview

The dataset consists of 2,395 nutrition items from `COMBINED_FOOD_DATASET.csv`.

### Data Transformation Process

```
Raw CSV → Parse nutritional data → Create Q&A pairs → JSON Lines format
```

Each food item is converted into a conversation:
- **User message:** Question about the food
- **Assistant message:** Nutritional information response

In [ ]:
# Load and explore the dataset
import pandas as pd
import json

# Load the CSV dataset
df = pd.read_csv('COMBINED_FOOD_DATASET.csv')

print(f"📊 Dataset Statistics:")
print(f"   Total items: {len(df)}")
print(f"   Columns: {list(df.columns)}")
print(f"\n📋 Sample data:")
print(df.head(3))

# Show how data is transformed
print("\n\n🔄 Example Transformation:")
print("=" * 70)

# Load a sample processed conversation
try:
    with open('data/processed/sample_nutrition_conversations.json', 'r') as f:
        sample = json.load(f)
    
    if isinstance(sample, list) and len(sample) > 0:
        example = sample[0]
        print("INPUT (from CSV):")
        print(f"  Food: {example.get('food_name', 'N/A')}")
        print("\nOUTPUT (conversational format):")
        print(json.dumps(example.get('messages', []), indent=2))
except FileNotFoundError:
    print("Sample file not found. Run the pipeline to generate it.")

# 5. Model Training with Vertex AI 🚀

## Pipeline Components

The training pipeline consists of 4 components:

### Component 1: Data Transformation
- **Input:** CSV file from GCS
- **Process:** Convert to conversational Q&A format
- **Output:** JSON Lines file split into train/test
- **Resources:** 4 CPUs, 8GB RAM

### Component 2: Fine-Tuning
- **Input:** Training data (JSON Lines)
- **Process:** Fine-tune Phi-3-mini with LoRA + 4-bit quantization
- **Output:** Fine-tuned model saved to GCS
- **Resources:** Tesla T4 GPU, 16 CPUs, 50GB RAM
- **Duration:** ~30-45 minutes

### Component 3: Inference
- **Input:** Test data + fine-tuned model
- **Process:** Generate predictions on test set
- **Output:** Predictions CSV
- **Resources:** Tesla T4 GPU, 8 CPUs, 32GB RAM

### Component 4: Evaluation
- **Input:** Predictions + ground truth
- **Process:** Compute BLEU and Rouge scores
- **Output:** Metrics JSON
- **Resources:** 4 CPUs, 8GB RAM

## Pipeline Execution Flow

```mermaid
graph LR
    A[Data Transform] --> B[Fine-Tune]
    B --> C[Inference]
    C --> D[Evaluation]
    B -.Model.-> C
```

In [ ]:
# Check latest pipeline run
from google.cloud import aiplatform

aiplatform.init(project='aerobic-polygon-460910-v9', location='europe-west2')

# List recent pipeline runs
print("📋 Recent Pipeline Runs:")
print("=" * 80)

try:
    pipeline_jobs = aiplatform.PipelineJob.list(
        filter='display_name:"nutrition*"',
        order_by='create_time desc'
    )
    
    for i, job in enumerate(list(pipeline_jobs)[:3]):  # Show last 3
        print(f"\n{i+1}. {job.display_name}")
        print(f"   State: {job.state.name}")
        print(f"   Created: {job.create_time}")
        print(f"   Job ID: {job.name.split('/')[-1]}")
        
        if job.state.name == 'PIPELINE_STATE_SUCCEEDED':
            print(f"   ✅ Status: SUCCEEDED")
        elif job.state.name == 'PIPELINE_STATE_RUNNING':
            print(f"   🔄 Status: RUNNING")
        else:
            print(f"   ❌ Status: {job.state.name}")
            
except Exception as e:
    print(f"Could not fetch pipeline runs: {e}")
    print("\n💡 Run the pipeline first:")
    print("   python scripts/pipeline_runner.py")

# 6. LoRA Fine-Tuning Configuration ⚙️

## What is LoRA?

**LoRA (Low-Rank Adaptation)** is a parameter-efficient fine-tuning method that:
- Freezes the base model weights
- Adds small trainable matrices to specific layers
- Reduces trainable parameters by ~99.8%
- Enables fine-tuning on consumer GPUs

## Our LoRA Configuration

| Parameter | Value | Purpose |
|-----------|-------|---------|
| **r (rank)** | 16 | Dimension of low-rank matrices |
| **lora_alpha** | 32 | Scaling factor (typically 2×r) |
| **lora_dropout** | 0.05 | Dropout for regularization |
| **Target Modules** | q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj | Which layers to adapt |

## Why 4-bit Quantization?

- **Reduces GPU memory:** 3.8B params × 4 bits = ~15GB (vs 60GB for FP16)
- **Enables training on T4 GPU:** 16GB VRAM
- **Uses NF4 (Normalized Float 4):** Optimal for neural network weights
- **Minimal accuracy loss:** <1% with proper configuration

In [ ]:
# Calculate trainable parameters
from src.constants import LORA_CONFIG

r = LORA_CONFIG['r']
lora_alpha = LORA_CONFIG['lora_alpha']

# Phi-3-mini has ~3.8B parameters
total_params = 3_800_000_000

# Estimate LoRA parameters (simplified calculation)
# For each target module: 2 matrices of size (hidden_dim, r)
# Phi-3 has hidden_dim=3072, and we target 7 modules per layer across ~32 layers
hidden_dim = 3072
num_modules_per_layer = 7
num_layers = 32

lora_params = 2 * hidden_dim * r * num_modules_per_layer * num_layers

print("📊 LoRA Parameter Analysis")
print("=" * 60)
print(f"Total Model Parameters:     {total_params:,}")
print(f"LoRA Trainable Parameters:  {lora_params:,}")
print(f"Percentage Trainable:       {(lora_params/total_params)*100:.2f}%")
print(f"\n💾 Memory Savings:")
print(f"Full Fine-tuning:           ~60 GB (FP16)")
print(f"LoRA + 4-bit:               ~15 GB")
print(f"Reduction:                  75%")
print("=" * 60)

# 7. Model Deployment to Vertex AI 🚢

## Deployment Architecture

```
Model (GCS) → Register to Model Registry → Deploy to Endpoint → Serve Predictions
```

## Deployment Configuration

| Setting | Value | Purpose |
|---------|-------|---------|
| **Model ID** | 3561348948692041728 | Registered model in Vertex AI |
| **Endpoint ID** | 5724492940806455296 | Production endpoint |
| **Machine Type** | n1-standard-8 | 8 vCPUs, 30GB RAM |
| **Accelerator** | NVIDIA Tesla T4 | GPU for inference |
| **Container** | HuggingFace TGI PyTorch | Pre-built serving container |
| **Min Replicas** | 1 | Always-on instance |
| **Max Replicas** | 1 | No auto-scaling (cost control) |

## Deployment Process

### Step 1: Register Model
The model artifact from the training pipeline is registered to Vertex AI Model Registry with a custom prediction handler.

### Step 2: Create/Get Endpoint
An endpoint is created (or retrieved if it exists) to serve predictions.

### Step 3: Deploy Model
The model is deployed to the endpoint with GPU configuration.

**⏱️ Deployment time:** 5-10 minutes

In [ ]:
# Check endpoint status
from google.cloud import aiplatform

aiplatform.init(project='aerobic-polygon-460910-v9', location='europe-west2')

ENDPOINT_ID = '5724492940806455296'

try:
    endpoint = aiplatform.Endpoint(f"projects/aerobic-polygon-460910-v9/locations/europe-west2/endpoints/{ENDPOINT_ID}")
    
    print("📍 ENDPOINT STATUS")
    print("=" * 70)
    print(f"Endpoint ID:       {ENDPOINT_ID}")
    print(f"Display Name:      {endpoint.display_name}")
    print(f"Resource Name:     {endpoint.resource_name}")
    print(f"Create Time:       {endpoint.create_time}")
    
    # Check deployed models
    deployed_models = endpoint.list_models()
    
    if deployed_models:
        print(f"\n✅ Models Deployed: {len(deployed_models)}")
        for model in deployed_models:
            print(f"\n   Model ID: {model.id}")
            print(f"   Display Name: {model.display_name}")
            print(f"   Machine Type: {model.machine_spec.machine_type}")
            print(f"   Accelerator: {model.machine_spec.accelerator_type}")
            print(f"   Accelerator Count: {model.machine_spec.accelerator_count}")
            print(f"   Traffic: {model.traffic_percentage}%")
    else:
        print(f"\n⚪ No models currently deployed")
        print(f"💰 Current cost: $0/hour")
        
except Exception as e:
    print(f"❌ Error checking endpoint: {e}")
    print("\n💡 Endpoint may not exist yet. Run:")
    print("   python scripts/deploy_to_endpoint.py")

# 8. 🚀 Launcher - Start/Stop Your Chatbot

## ▶️ START THE CHATBOT (3 Steps)

### Step 1: Deploy Model to Endpoint

```powershell
python scripts/deploy_to_endpoint.py
```

⏱️ **Wait:** 5-10 minutes for deployment to complete

### Step 2: Verify Deployment

```powershell
python scripts/check_endpoint_status.py
```

✅ **Look for:** "DEPLOYMENT COMPLETE!" and Status: "SERVING"

### Step 3: Launch Web Interface

```powershell
.\.venv\Scripts\chainlit.exe run src/app/main.py -w
```

🌐 **Opens at:** http://localhost:8000

---

## ⏹️ STOP EVERYTHING (Save Money!)

### Step 1: Stop Chatbot Interface

Press `Ctrl+C` in the terminal running Chainlit

### Step 2: Undeploy Model (STOPS BILLING!)

```powershell
python scripts/undeploy_model.py
```

💰 **This immediately stops hourly charges!**

### Step 3: Verify Model is Undeployed

```powershell
python scripts/check_endpoint_status.py
```

✅ **Should show:** "Status: No models deployed"

---

## 🔗 Monitoring Links

- **Endpoint Status:** [View in GCP Console](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints/5724492940806455296?project=aerobic-polygon-460910-v9)
- **Pipeline History:** [View Training Runs](https://console.cloud.google.com/vertex-ai/pipelines?project=aerobic-polygon-460910-v9)
- **Storage Bucket:** [View GCS Files](https://console.cloud.google.com/storage/browser/llmops_101_europ?project=aerobic-polygon-460910-v9)

---

## 🖱️ Manual Deployment via GCP Console

If you prefer to deploy through the web interface:

### Settings to Use:

| Setting | Value |
|---------|-------|
| **Model Name** | nutrition-assistant-phi3 |
| **Model ID** | 3561348948692041728 |
| **Deployed Name** | nutrition-assistant-deployed |
| **Machine Type** | n1-standard-8 |
| **Accelerator** | NVIDIA_TESLA_T4 |
| **Accelerator Count** | 1 |
| **Min Nodes** | 1 |
| **Max Nodes** | 1 |
| **Traffic Split** | 100 |

In [ ]:
# You can run these commands directly from the notebook (not recommended - use terminal instead)
# This cell demonstrates what happens when you check status

import subprocess

print("🔍 Checking Endpoint Status...")
print("=" * 70)

try:
    result = subprocess.run(
        ['python', 'scripts/check_endpoint_status.py'],
        capture_output=True,
        text=True,
        timeout=30
    )
    
    print(result.stdout)
    if result.stderr:
        print("Errors:", result.stderr)
        
except subprocess.TimeoutExpired:
    print("⏱️ Command timed out")
except Exception as e:
    print(f"❌ Error: {e}")
    
print("\n💡 TIP: Run these commands in your terminal for better output")

# 9. Custom Handler Implementation 🛠️

The custom handler (`src/handler.py`) is crucial for deployment. It tells Vertex AI how to:
1. Load the fine-tuned model
2. Process prediction requests
3. Return formatted responses

## Handler Structure

```python
class EndpointHandler:
    def __init__(self, model_dir):
        # Load the fine-tuned model with LoRA adapters
        # Apply 4-bit quantization
        # Load tokenizer
        
    def __call__(self, request):
        # Extract input text
        # Tokenize
        # Generate response
        # Return JSON
```

## Key Features

- **4-bit Quantization:** Loads model in NF4 format to fit in GPU memory
- **LoRA Adapters:** Applies fine-tuned weights on top of base model
- **Streaming Support:** Can return responses token-by-token
- **Error Handling:** Graceful fallbacks for invalid inputs

In [ ]:
# View the handler implementation
with open('src/handler.py', 'r') as f:
    handler_code = f.read()
    
print("📄 Custom Handler Code (src/handler.py)")
print("=" * 70)
print(handler_code[:1500])  # Show first 1500 characters
print("\n... (truncated)")
print("\n💡 Full file: src/handler.py")

# 10. Cost Management & Monitoring 💰

## Cost Breakdown

| Status | Component | Cost/Hour | Cost/Day (8hrs) | Cost/Month |
|--------|-----------|-----------|-----------------|------------|
| ⚪ **Undeployed** | Endpoint (empty) | $0.00 | $0.00 | $0.00 |
| ⚪ **Undeployed** | GCS Storage | ~$0.001 | ~$0.008 | ~$0.72 |
| | **TOTAL UNDEPLOYED** | **$0.001** | **$0.008** | **~$0.72** |
| | | | | |
| ✅ **Deployed** | n1-standard-8 | ~$0.38 | ~$3.04 | ~$274 |
| ✅ **Deployed** | Tesla T4 GPU | ~$0.35 | ~$2.80 | ~$252 |
| ✅ **Deployed** | Endpoint overhead | ~$0.02 | ~$0.16 | ~$14 |
| | **TOTAL DEPLOYED** | **~$0.75** | **~$6.00** | **~$540** |

## Cost Optimization Best Practices

1. **✅ Always undeploy after testing**
   ```powershell
   python scripts/undeploy_model.py
   ```

2. **✅ Check status before leaving**
   ```powershell
   python scripts/check_endpoint_status.py
   ```

3. **✅ Set up billing alerts in GCP Console**
   - Go to: Billing → Budgets & alerts
   - Set threshold: $10/month
   - Get email notifications

4. **❌ Don't leave models deployed overnight**
   - 8 hours unused = ~$6 wasted
   - 1 month unused = ~$540!

## Monitoring Dashboard

In [ ]:
# Calculate estimated costs based on current deployment status
from google.cloud import aiplatform
from datetime import datetime, timedelta

aiplatform.init(project='aerobic-polygon-460910-v9', location='europe-west2')

ENDPOINT_ID = '5724492940806455296'

try:
    endpoint = aiplatform.Endpoint(f"projects/aerobic-polygon-460910-v9/locations/europe-west2/endpoints/{ENDPOINT_ID}")
    deployed_models = endpoint.list_models()
    
    print("💰 CURRENT COST ANALYSIS")
    print("=" * 70)
    
    if deployed_models:
        print("⚠️  MODEL IS DEPLOYED - BILLING ACTIVE")
        print("\nHourly Cost Breakdown:")
        print("  n1-standard-8 (8 vCPUs, 30GB RAM): $0.38/hour")
        print("  Tesla T4 GPU:                      $0.35/hour")
        print("  Endpoint overhead:                 $0.02/hour")
        print("  " + "-" * 50)
        print("  TOTAL:                             $0.75/hour")
        
        # Estimate time deployed
        create_time = endpoint.create_time
        if create_time:
            hours_deployed = (datetime.now(create_time.tzinfo) - create_time).total_seconds() / 3600
            estimated_cost = hours_deployed * 0.75
            print(f"\nEstimated cost since creation:     ${estimated_cost:.2f}")
            
        print("\n⚠️  ACTION REQUIRED: Run `python scripts/undeploy_model.py` to stop billing!")
    else:
        print("✅ MODEL IS UNDEPLOYED - MINIMAL BILLING")
        print("\nCurrent costs:")
        print("  GCS Storage only:                  $0.001/hour")
        print("  Daily cost:                        $0.024")
        print("  Monthly cost:                      ~$0.72")
        print("\n✅ You're good! No action needed.")
        
except Exception as e:
    print(f"Could not check costs: {e}")

# 11. Pipeline Components Deep Dive 🔬

## Component Files

All pipeline components are in `src/pipeline_components/`:

### 1. data_transformation_component.py
**Purpose:** Transform CSV to conversational format

**Key Functions:**
- `transform_data()`: Main transformation logic
- `create_conversation()`: Converts food item to Q&A pair
- `split_train_test()`: 80/20 split

**Inputs:**
- `dataset_path`: Path to CSV in GCS

**Outputs:**
- `train_data`: Training JSON Lines
- `test_data`: Test JSON Lines

**Resources:**
- CPU: 4
- RAM: 8GB
- Disk: 10GB

---

### 2. fine_tuning_component.py
**Purpose:** Fine-tune Phi-3 with LoRA

**Key Functions:**
- `load_model()`: Load base model with 4-bit quantization
- `apply_lora()`: Add LoRA adapters
- `train()`: Training loop with Transformers Trainer

**Inputs:**
- `train_data`: Training dataset
- `base_model_id`: "microsoft/Phi-3-mini-4k-instruct"

**Outputs:**
- `model_dir`: Fine-tuned model in GCS

**Resources:**
- GPU: Tesla T4 (16GB VRAM)
- CPU: 16
- RAM: 50GB
- Duration: ~30-45 minutes

---

### 3. inference_component.py
**Purpose:** Generate predictions on test set

**Key Functions:**
- `load_model()`: Load fine-tuned model
- `predict()`: Generate responses
- `batch_predict()`: Process multiple inputs

**Inputs:**
- `model_dir`: Fine-tuned model
- `test_data`: Test dataset

**Outputs:**
- `predictions`: CSV with generated responses

**Resources:**
- GPU: Tesla T4
- CPU: 8
- RAM: 32GB

---

### 4. evaluation_component.py
**Purpose:** Compute metrics (BLEU, Rouge)

**Key Functions:**
- `compute_bleu()`: BLEU score calculation
- `compute_rouge()`: Rouge score calculation
- `aggregate_metrics()`: Summary statistics

**Inputs:**
- `predictions`: Model predictions
- `ground_truth`: Reference answers

**Outputs:**
- `metrics`: JSON with scores
- `per_sample_metrics`: Detailed results

**Resources:**
- CPU: 4
- RAM: 8GB

In [ ]:
# List all component files
import os

components_dir = 'src/pipeline_components'
print("📁 Pipeline Component Files:")
print("=" * 70)

for filename in os.listdir(components_dir):
    if filename.endswith('.py') and not filename.startswith('__'):
        filepath = os.path.join(components_dir, filename)
        # Get file size
        size = os.path.getsize(filepath)
        print(f"\n✅ {filename}")
        print(f"   Size: {size:,} bytes ({size//1024} KB)")
        
        # Count lines
        with open(filepath, 'r') as f:
            lines = len(f.readlines())
        print(f"   Lines of code: {lines}")

print("\n" + "=" * 70)
print("💡 View full code in: src/pipeline_components/")

# 12. Troubleshooting Guide 🔧

## Common Issues and Solutions

### ❌ Issue 1: Authentication Error

**Symptom:**
```
Error: Could not automatically determine credentials
```

**Solution:**
```powershell
gcloud auth application-default login
```

**Why it happens:** Your local credentials expired or weren't set up

---

### ❌ Issue 2: GPU Quota Error

**Symptom:**
```
Quota 'NVIDIA_T4_GPUS' exceeded
```

**Solution:**
1. Go to: [GCP Console → Quotas](https://console.cloud.google.com/iam-admin/quotas)
2. Filter: "Vertex AI" + "europe-west2" + "NVIDIA T4"
3. Request quota increase to 1
4. Wait for approval (usually 1-2 hours)

---

### ❌ Issue 3: Model Not Responding

**Symptom:**
Chainlit shows errors or no response

**Solution:**
```powershell
# 1. Check endpoint status
python scripts/check_endpoint_status.py

# 2. If not SERVING, check deployment
# Look for errors in GCP Console

# 3. If needed, redeploy
python scripts/undeploy_model.py
python scripts/deploy_to_endpoint.py
```

---

### ❌ Issue 4: Pipeline Fails at Fine-Tuning

**Symptom:**
Pipeline shows "FAILED" status on fine-tuning component

**Checklist:**
- ✅ GPU quota approved?
- ✅ Dataset uploaded to GCS?
- ✅ Correct region (europe-west2)?
- ✅ Bucket permissions correct?

**View logs:**
```powershell
python scripts/check_pipeline_status.py
```

Or in GCP Console → Vertex AI → Pipelines

---

### ❌ Issue 5: Chainlit Won't Start

**Symptom:**
```
chainlit: command not found
```

**Solution:**
```powershell
# Use python -m instead
python -m chainlit run src/app/main.py -w

# Or reinstall
pip install chainlit google-auth
```

---

### ❌ Issue 6: High GCP Costs

**Symptom:**
Unexpected billing charges

**Solution:**
```powershell
# 1. Check what's deployed
python scripts/check_endpoint_status.py

# 2. Undeploy immediately
python scripts/undeploy_model.py

# 3. Verify it's undeployed
python scripts/check_endpoint_status.py
```

**Prevention:**
- Always undeploy after testing
- Set up billing alerts at $10/month
- Check status before leaving

In [ ]:
# Quick diagnostic check
import subprocess
import os

print("🔍 QUICK DIAGNOSTIC CHECK")
print("=" * 70)

# Check 1: Python environment
print("\n1. Python Environment:")
import sys
print(f"   ✅ Python {sys.version.split()[0]}")
print(f"   ✅ Virtual env: {sys.prefix}")

# Check 2: GCP Authentication
print("\n2. GCP Authentication:")
try:
    import google.auth
    creds, project = google.auth.default()
    print(f"   ✅ Authenticated")
    print(f"   ✅ Project: {project}")
except Exception as e:
    print(f"   ❌ Not authenticated: {e}")

# Check 3: Required files exist
print("\n3. Project Files:")
required_files = [
    'src/handler.py',
    'src/app/main.py',
    'src/constants.py',
    'COMBINED_FOOD_DATASET.csv'
]

for file in required_files:
    if os.path.exists(file):
        print(f"   ✅ {file}")
    else:
        print(f"   ❌ {file} NOT FOUND")

# Check 4: Scripts
print("\n4. Deployment Scripts:")
scripts = [
    'scripts/deploy_to_endpoint.py',
    'scripts/undeploy_model.py',
    'scripts/check_endpoint_status.py'
]

for script in scripts:
    if os.path.exists(script):
        print(f"   ✅ {script}")
    else:
        print(f"   ❌ {script} NOT FOUND")

print("\n" + "=" * 70)
print("✅ Diagnostic complete!")

# 📚 Complete Workflow Summary

## End-to-End Process

### Phase 1: Setup (One-Time)
1. Clone repository
2. Install dependencies with `uv sync`
3. Authenticate with GCP: `gcloud auth application-default login`
4. Validate setup: `python scripts/validate_gcp_setup.py`

### Phase 2: Training (Optional - Already Done!)
1. Upload dataset: `python scripts/upload_dataset.py`
2. Run pipeline: `python scripts/pipeline_runner.py`
3. Monitor: `python scripts/check_pipeline_status.py`
4. **Result:** Model saved to GCS with ID `3561348948692041728`

### Phase 3: Deployment (When Needed)
1. Deploy model: `python scripts/deploy_to_endpoint.py`
2. Wait 5-10 minutes
3. Verify: `python scripts/check_endpoint_status.py`
4. **Result:** Endpoint `5724492940806455296` serving requests

### Phase 4: Usage
1. Launch chatbot: `.\.venv\Scripts\chainlit.exe run src/app/main.py -w`
2. Open browser: http://localhost:8000
3. Chat with your nutrition assistant!

### Phase 5: Cleanup (IMPORTANT!)
1. Stop chatbot: `Ctrl+C`
2. Undeploy model: `python scripts/undeploy_model.py`
3. Verify: `python scripts/check_endpoint_status.py`
4. **Result:** $0/hour billing

---

## 🎓 Key Learnings

### What You Built
- ✅ Complete MLOps pipeline with Vertex AI
- ✅ LoRA fine-tuning with 4-bit quantization
- ✅ Production deployment with GPU inference
- ✅ Web interface for real users
- ✅ Cost management automation

### Technologies Mastered
- Google Cloud Platform (Vertex AI, GCS)
- Kubeflow Pipelines
- Hugging Face Transformers
- LoRA (Parameter-Efficient Fine-Tuning)
- Chainlit (Chat UI framework)
- Docker containers
- Python packaging

### Production Skills
- Cost optimization
- Monitoring and logging
- Error handling
- Authentication and security
- CI/CD principles

---

## 📖 Additional Resources

### Documentation
- **Main Guide:** `docs/guides/HOW_TO_LAUNCH.md`
- **Quick Reference:** `docs/guides/QUICK_REFERENCE.md`
- **French Guide:** `docs/guides/GUIDE_LANCEMENT_FR.md`
- **Project Summary:** `docs/references/PROJECT_COMPLETE.md`

### GCP Console Links
- [Vertex AI Endpoints](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints/5724492940806455296?project=aerobic-polygon-460910-v9)
- [Pipeline Runs](https://console.cloud.google.com/vertex-ai/pipelines?project=aerobic-polygon-460910-v9)
- [GCS Bucket](https://console.cloud.google.com/storage/browser/llmops_101_europ?project=aerobic-polygon-460910-v9)

### External Resources
- [Vertex AI Documentation](https://cloud.google.com/vertex-ai/docs)
- [Phi-3 Model Card](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)
- [LoRA Paper](https://arxiv.org/abs/2106.09685)
- [Chainlit Documentation](https://docs.chainlit.io)

---

## 🎉 Project Status

**✅ COMPLETE & PRODUCTION READY**

- Model trained: 2,395 nutrition items
- Endpoint created: `5724492940806455296`
- Model registered: `3561348948692041728`
- Chatbot ready: `src/app/main.py`
- Documentation complete: All guides written
- Cost management: Deploy/undeploy automation

**Current Billing:** $0/hour (model undeployed)

---

**Made with ❤️ for Albert School LLM OPS Bootcamp MSC2**  
**Completed:** October 21, 2025

🥗 **Your AI Nutrition Assistant is Ready!**